In [11]:


#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
df = pd.read_pickle('dados_juridicos.pkl')
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=72,embedding_column_name='embedding')
centroids = calculate_hdbscan_centroids(df,n_components=n_components)
plot_clusters_umap(df)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 2
query_text = "cidade de alvorada"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)

best_chunks_info, best_similarities = find_chunks_in_clusters_hdbscan(centroids, query_umap,  df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
#plot_query_2_components(query_umap,fig,query_text,plot=True)

ModuleNotFoundError: No module named 'search_functions'

In [6]:
import json
import pandas as pd

# Replace 'your_file_path.json' with the actual path to your JSON file
file_path = 'brquad-gte-dev-v2.0.json'

# Reading the JSON data from the file
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extracting context and question from each QA object
contexts_questions = [
    {'context': paragraph['context'], 'query': qa['question']}
    for item in data['data']
    for paragraph in item['paragraphs']
    for qa in paragraph['qas']
]

# Creating a DataFrame from the extracted data
df = pd.DataFrame(contexts_questions)

# The DataFrame 'df' now contains two columns: 'context' and 'query'
# You can use df.head() to print the first few rows of the DataFrame
print(df.head())
df.to_pickle('brquad-gte-dev-v2.0.pkl')


                                             context  \
0  Os normandos (normandos: nourmands; francês: n...   
1  Os normandos (normandos: nourmands; francês: n...   
2  Os normandos (normandos: nourmands; francês: n...   
3  Os normandos (normandos: nourmands; francês: n...   
4  Os normandos (normandos: nourmands; francês: n...   

                                               query  
0           Em que país a Normandia está localizada?  
1          Quando os normandos estavam na Normandia?  
2         De quais países os nórdicos se originaram?  
3                          Quem era o líder nórdico?  
4  Em que século os normandos ganharam sua identi...  


In [3]:
import pandas as pd
df = pd.read_pickle('brquad-gte-dev-v2.0.pkl')
from openai_utils import *
df['embedding'] = df['context'].apply(lambda x: embed_with_openai(x))
df.to_pickle('brquad-gte-dev-v2.0_embedded.pkl')

In [6]:
import pandas as pd
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded.pkl')

In [8]:
import json
import pandas as pd

# Assuming your existing DataFrame is named df and created from the earlier code
# Replace 'your_file_path.json' with the actual path to your JSON file
file_path = 'brquad-gte-dev-v2.0.json'

# Reading the JSON data from the file again to add the document names
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extracting document names (titles) for each context-question pair
document_names = [
    item['title']
    for item in data['data']
    for paragraph in item['paragraphs']
    for _ in paragraph['qas']  # Use underscore since we only need the count of QAs to match titles
]

# Assuming the lengths of df and document_names match
df['document name'] = document_names

# Now df contains three columns: 'context', 'query', and 'document name'
print(df.head())

df.to_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')


                                             context  \
0  Os normandos (normandos: nourmands; francês: n...   
1  Os normandos (normandos: nourmands; francês: n...   
2  Os normandos (normandos: nourmands; francês: n...   
3  Os normandos (normandos: nourmands; francês: n...   
4  Os normandos (normandos: nourmands; francês: n...   

                                               query  \
0           Em que país a Normandia está localizada?   
1          Quando os normandos estavam na Normandia?   
2         De quais países os nórdicos se originaram?   
3                          Quem era o líder nórdico?   
4  Em que século os normandos ganharam sua identi...   

                                           embedding document name  
0  [0.02337769791483879, 0.06848494708538055, -0....       Normans  
1  [0.02337769791483879, 0.06848494708538055, -0....       Normans  
2  [0.02337769791483879, 0.06848494708538055, -0....       Normans  
3  [0.02337769791483879, 0.06848494708538055, -0..

In [2]:

#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)

#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
pca, normalizer,n_components,df  = create_pca(df,n_components=4,embedding_column_name='embedding')
centroids = create_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_components(df,plot=False)
distance_threshold = 0.04
query_text = "Em que país a Normandia está localizada?"
query_embedding = create_query_embedding(query_text)
query_pca = create_query_pca(pca,normalizer,query_embedding)
best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_pca, query_embedding, df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
#plot_query_2_components(query_pca,fig,query_text,plot=True)

Distance from query to centroid of 'Victoria_ (Austrália)': 0.21660393193025346
Distance from query to centroid of 'Normans': 0.2224797874245965
Distance from query to centroid of 'Rhine': 0.2397687807214243
Distance from query to centroid of 'Jacksonville, Flórida': 0.2407349961924958
Distance from query to centroid of 'huguenote': 0.25165649685733793
Distance from query to centroid of 'Varsóvia': 0.25785548810382397
Distance from query to centroid of 'Black_Death': 0.27395694751495575
Distance from query to centroid of 'Guerra da França e da Índia': 0.2990610737061827
Distance from query to centroid of 'Floresta amazônica': 0.3637067395695744
Distance from query to centroid of 'Fresno, Califórnia': 0.3796470443297164
Distance from query to centroid of 'Sul da Califórnia': 0.3877285230174726
Distance from query to centroid of 'Interrupção de pacote': 0.39494828073900495
Distance from query to centroid of 'Farmacia': 0.4023920566430823
Distance from query to centroid of 'Ctenophora': 0

In [4]:

#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)

#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=4,embedding_column_name='embedding')
centroids = create_centroids_umap(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_components(df,plot=False)
distance_threshold = 0.04
query_text = "Em que país a Normandia está localizada?"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(umap,normalizer,query_embedding)
best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_umap, query_embedding, df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
#plot_query_2_components(query_pca,fig,query_text,plot=True)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


AttributeError: 'list' object has no attribute 'transform'

In [2]:


#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=38,embedding_column_name='embedding')
centroids = calculate_hdbscan_centroids(df,n_components=n_components)
#plot_clusters_umap(df)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 10
query_text = "Em que país a Normandia está localizada?"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)

best_chunks_info, best_similarities = find_chunks_in_clusters_hdbscan(centroids, query_umap,  df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
#plot_query_2_components(query_pca,fig,query_text,plot=True)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Distance from query to centroid of cluster '101': 6.233075141906738
Distance from query to centroid of cluster '111': 6.606701374053955
Distance from query to centroid of cluster '12': 7.4573140144348145
Distance from query to centroid of cluster '115': 7.488110542297363
Distance from query to centroid of cluster '114': 8.11187744140625
Distance from query to centroid of cluster '62': 8.318522453308105
Distance from query to centroid of cluster '14': 9.039209365844727
Distance from query to centroid of cluster '108': 9.234598159790039
Distance from query to centroid of cluster '59': 9.559696197509766
Distance from query to centroid of cluster '10': 9.604107856750488
Distance from query to centroid of cluster '102': 9.84427261352539
Distance from query to centroid of cluster '218': 10.149617195129395
Distance from query to centroid of cluster '103': 12.080317497253418
Distance from query to centroid of cluster '243': 12.658280372619629
Distance from query to centroid of cluster '56': 13

In [1]:


#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=12,embedding_column_name='embedding')
centroids = calculate_hdbscan_centroids(df,n_components=n_components)
#plot_clusters_umap(df)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 3
query_text = "Em que país a Normandia está localizada?"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)


#best_chunks_info, best_similarities = find_chunks_in_clusters_hdbscan(centroids, query_umap,  df,top_k = 5,reverse_order = False, distance_threshold=distance_threshold,embedding_column_name='embedding')
query_umap = query_umap.flatten()
best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_umap, query_embedding, df,distance_threshold=distance_threshold,embedding_column_name='embedding')

show_findings(best_chunks_info, best_similarities)
#plot_query_2_components(query_pca,fig,query_text,plot=True)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


Distance from query to centroid of '168': 4.692285537719727
Distance from query to centroid of '165': 4.971076011657715
Distance from query to centroid of '125': 5.785287857055664
Distance from query to centroid of '169': 5.868828296661377
Distance from query to centroid of '189': 6.367888927459717
Distance from query to centroid of '54': 6.6391801834106445
Distance from query to centroid of '17': 6.910887241363525
Distance from query to centroid of '191': 6.920158863067627
Distance from query to centroid of '56': 6.990875244140625
Distance from query to centroid of '192': 7.2561140060424805
Distance from query to centroid of '33': 7.3502397537231445
Distance from query to centroid of '159': 7.475391864776611
Distance from query to centroid of '184': 8.462628364562988
Distance from query to centroid of '158': 8.542115211486816
Distance from query to centroid of '75': 8.812493324279785
Distance from query to centroid of '9': 8.864561080932617
Distance from query to centroid of '157': 9.

In [6]:
#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=512,embedding_column_name='embedding')
centroids = calculate_hdbscan_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 10
query_text = "Qual é a força exercida pela gravidade padrão em uma tonelada de massa?"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)

best_chunks_info, best_similarities = find_chunks_in_clusters_hdbscan(centroids, query_umap,  df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Distance from query to centroid of cluster '139': 0.0299562718719244
Distance from query to centroid of cluster '212': 13.457308769226074
Distance from query to centroid of cluster '232': 13.616426467895508
Distance from query to centroid of cluster '240': 13.732830047607422
Distance from query to centroid of cluster '241': 13.737948417663574
Distance from query to centroid of cluster '235': 13.74106216430664
Distance from query to centroid of cluster '238': 13.955266952514648
Distance from query to centroid of cluster '209': 13.956266403198242
Distance from query to centroid of cluster '231': 14.026378631591797
Distance from query to centroid of cluster '236': 14.030989646911621
Distance from query to centroid of cluster '190': 14.104610443115234
Distance from query to centroid of cluster '106': 14.12998104095459
Distance from query to centroid of cluster '221': 14.135977745056152
Distance from query to centroid of cluster '219': 14.161476135253906
Distance from query to centroid of c

In [7]:
#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=1024,embedding_column_name='embedding')
centroids = calculate_hdbscan_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 10
query_text = "Qual é a força exercida pela gravidade padrão em uma tonelada de massa?"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)

best_chunks_info, best_similarities = find_chunks_in_clusters_hdbscan(centroids, query_umap,  df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
f:\woopi\clusterhdb\testeste\umap_methods.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'umap component {i+1}'] = umapComponents[:, i]
f:\woopi\clusterhdb\testeste\umap_methods.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

Distance from query to centroid of cluster '71': 0.0014635297702625394
Distance from query to centroid of cluster '86': 16.79277992248535
Distance from query to centroid of cluster '87': 17.389474868774414
Distance from query to centroid of cluster '85': 17.593515396118164
Distance from query to centroid of cluster '81': 17.915090560913086
Distance from query to centroid of cluster '84': 18.045372009277344
Distance from query to centroid of cluster '80': 18.116535186767578
Distance from query to centroid of cluster '75': 18.147216796875
Distance from query to centroid of cluster '82': 18.27985954284668
Distance from query to centroid of cluster '88': 18.429637908935547
Distance from query to centroid of cluster '89': 18.483787536621094
Distance from query to centroid of cluster '76': 18.58266830444336
Distance from query to centroid of cluster '74': 18.59461784362793
Distance from query to centroid of cluster '83': 18.76311683654785
Distance from query to centroid of cluster '78': 18.7

In [2]:

#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from openai_utils import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)

#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
pca, normalizer,n_components,df  = create_pca(df,n_components=4,embedding_column_name='embedding')
centroids = create_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_components(df,plot=False)
distance_threshold = 0.05
query_text = "Quem identificou a gravidade como uma força?"
query_embedding = create_query_embedding(query_text)
query_pca = create_query_pca(pca,normalizer,query_embedding)
best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_pca, query_embedding, df,top_contexts = 2, distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
contexts = [chunk['chunk_text'] for chunk in best_chunks_info]
for context in contexts:
    response = ask_with_openai(context,query_text)
    print(response)
    

#plot_query_2_components(query_pca,fig,query_text,plot=True)

f:\woopi\clusterhdb\testeste\cluster_functions.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'principal component {i+1}'] = principalComponents[:, i]
f:\woopi\clusterhdb\testeste\cluster_functions.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'principal component {i+1}'] = principalComponents[:, i]
f:\woopi\clusterhdb\testeste\cluster_functions.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

Distance from query to centroid of 'Número primo': 0.16666772270710103
Distance from query to centroid of 'Força': 0.1757150143767068
Distance from query to centroid of 'Geologia': 0.17937673053226327
Distance from query to centroid of 'Oxigênio': 0.1920965176644251
Distance from query to centroid of 'Steam_engine': 0.22410429518461283
Distance from query to centroid of 'Sistema imunológico': 0.22958309659005088
Distance from query to centroid of 'Ctenophora': 0.24753371798368518
Distance from query to centroid of 'Teoria_complexidade_computacional': 0.29593734989460696
Distance from query to centroid of 'Painel Intergovernamental de Mudanças Climáticas': 0.304825955596892
Distance from query to centroid of 'Interrupção de pacote': 0.3315872719222562
Distance from query to centroid of 'Farmacia': 0.3362514485642327
Distance from query to centroid of 'Imperialismo': 0.34234518839455447
Distance from query to centroid of 'Black_Death': 0.34664077125266946
Distance from query to centroid 

In [4]:

#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from openai_utils import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
import warnings

# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
pca, normalizer,n_components,df  = create_pca(df,n_components=127,embedding_column_name='embedding')
centroids = create_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_components(df,plot=False)
distance_threshold = 0.1
query_text = "Na visão de Galileu, a esfera terrestre continha quantos elementos?"
query_embedding = create_query_embedding(query_text)
query_pca = create_query_pca(pca,normalizer,query_embedding)
best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_pca, query_embedding, df,top_contexts = 10, distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
contexts = [chunk['chunk_text'] for chunk in best_chunks_info]
#for context in contexts:
#    response = ask_with_openai(context,query_text)
#    print(response)
response = ask_with_openai(contexts[0],query_text)
print("Pergunta: ",query_text)
print("Resposta: ",response)    

#plot_query_2_components(query_pca,fig,query_text,plot=True)

Distance from query to centroid of 'Força': 0.7373450464189006
Distance from query to centroid of 'Geologia': 0.7500236182870894
Distance from query to centroid of 'Oxigênio': 0.763396039428479
Distance from query to centroid of 'Número primo': 0.8191799723900642
Distance from query to centroid of 'Painel Intergovernamental de Mudanças Climáticas': 0.8706618517769225
Distance from query to centroid of 'Imperialismo': 0.8775279267949962
Distance from query to centroid of 'Floresta amazônica': 0.8908505514642806
Distance from query to centroid of 'European_Union_law': 0.8917819176685323
Distance from query to centroid of 'University_of_Chicago': 0.8996842780338614
Distance from query to centroid of 'Universidade de Harvard': 0.9010848240737941
Distance from query to centroid of 'Ctenophora': 0.9029404100031265
Distance from query to centroid of 'Teoria_complexidade_computacional': 0.9174941089385392
Distance from query to centroid of 'Sistema imunológico': 0.9298570202599916
Distance fro

In [2]:

#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from openai_utils import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
import warnings

# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
pca, normalizer,n_components,df  = create_pca(df,n_components=127,embedding_column_name='embedding')
centroids = create_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_components(df,plot=False)
distance_threshold = 0.1
query_text = "Como Aristóteles se referiu ao movimento forçado?"
query_embedding = create_query_embedding(query_text)
query_pca = create_query_pca(pca,normalizer,query_embedding)
best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_pca, query_embedding, df,top_contexts = 10, distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
contexts = [chunk['chunk_text'] for chunk in best_chunks_info]
#for context in contexts:
#    response = ask_with_openai(context,query_text)
#    print(response)
response = ask_with_openai(contexts[0],query_text)
print("Pergunta: ",query_text)
print("Resposta: ",response)    

#plot_query_2_components(query_pca,fig,query_text,plot=True)

Distance from query to centroid of 'Força': 0.5628108293184882
Distance from query to centroid of 'Desobediência civil': 0.8033930600518611
Distance from query to centroid of 'Oxigênio': 0.8050158893903613
Distance from query to centroid of 'Geologia': 0.8113134909376282
Distance from query to centroid of 'Imperialismo': 0.8119538604352106
Distance from query to centroid of 'Steam_engine': 0.8199168007752937
Distance from query to centroid of 'Islamismo': 0.8464149024378034
Distance from query to centroid of 'Teoria_complexidade_computacional': 0.856509743672131
Distance from query to centroid of 'Ctenophora': 0.8765969316880514
Distance from query to centroid of 'Universidade de Harvard': 0.894704317312837
Distance from query to centroid of 'Sistema imunológico': 0.8974505748589072
Distance from query to centroid of 'Desigualdade econômica': 0.9061641848433708
Distance from query to centroid of 'European_Union_law': 0.91332282888517
Distance from query to centroid of '1973_oil_crisis'

In [1]:

#documentos que mencionam policia civil
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from openai_utils import *


import warnings
# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)


#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
pca, normalizer,n_components,df  = create_pca(df,n_components=127,embedding_column_name='embedding')
centroids = create_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_components(df,plot=False)
distance_threshold = 0.1
query_text = "Onde Aristóteles acreditava no lugar natural para os elementos da terra e da água?"
query_embedding = create_query_embedding(query_text)
query_pca = create_query_pca(pca,normalizer,query_embedding)
best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_pca, query_embedding, df,top_contexts = 10, distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)
contexts = [chunk['chunk_text'] for chunk in best_chunks_info]
#for context in contexts:
#    response = ask_with_openai(context,query_text)
#    print(response)
response = ask_with_openai(contexts[0],query_text)
print("Pergunta: ",query_text)
print("Resposta: ",response)    

#plot_query_2_components(query_pca,fig,query_text,plot=True)

Distance from query to centroid of 'Geologia': 0.7203878785398544
Distance from query to centroid of 'Oxigênio': 0.7222049971680122
Distance from query to centroid of 'Força': 0.7735732768142067
Distance from query to centroid of 'Floresta amazônica': 0.8494345073799311
Distance from query to centroid of 'Imperialismo': 0.8603821407483875
Distance from query to centroid of 'Ctenophora': 0.8875500258245786
Distance from query to centroid of 'Rhine': 0.8907474518361629
Distance from query to centroid of 'Painel Intergovernamental de Mudanças Climáticas': 0.8950440806279039
Distance from query to centroid of 'Universidade de Harvard': 0.9032513176135928
Distance from query to centroid of 'Número primo': 0.9084082954497633
Distance from query to centroid of 'University_of_Chicago': 0.9190181550166323
Distance from query to centroid of 'Farmacia': 0.9260548893549198
Distance from query to centroid of 'Steam_engine': 0.9273699812860833
Distance from query to centroid of 'Normans': 0.92858531

In [3]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *

import warnings
# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

####faz sentido testar o hdbscan aqui?? muitos documentos variados que já estão separados pelo nome do documento


df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=32,embedding_column_name='embedding')
centroids = calculate_hdbscan_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 20
query_text = "Normandia"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)

best_chunks_info, best_similarities = find_chunks_in_clusters_hdbscan(centroids, query_umap,  df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Distance from query to centroid of cluster '842': 0.07049062103033066
Distance from query to centroid of cluster '843': 0.0705816000699997
Distance from query to centroid of cluster '285': 6.975915431976318
Distance from query to centroid of cluster '1073': 7.361433982849121
Distance from query to centroid of cluster 'noise_Guerra da França e da Índia': 7.417236328125
Distance from query to centroid of cluster '52': 7.5539631843566895
Distance from query to centroid of cluster '477': 7.5911431312561035
Distance from query to centroid of cluster '1115': 7.661538600921631
Distance from query to centroid of cluster '1349': 7.728135108947754
Distance from query to centroid of cluster '1345': 7.9106316566467285
Distance from query to centroid of cluster '1346': 7.910675048828125
Distance from query to centroid of cluster '1278': 7.910736560821533
Distance from query to centroid of cluster '1419': 8.1011323928833
Distance from query to centroid of cluster '1454': 8.402295112609863
Distance f

In [2]:

filtro = df.loc[df['cluster_label'] == -1]

###Ideia:
#tratar os documentos que não foram clusterizados como um cluster separado e separar por documento

# Imprime o resultado do filtro
print(filtro)


                                              chunk_text  \
34     Os descendentes dos vikings de Rollo e suas es...   
39     Os normandos adotaram posteriormente as cresce...   
55     Alguns normandos uniram forças turcas para aju...   
80     O declínio adicional do estado de coisas bizan...   
102    Em 1066, o duque Guilherme II da Normandia con...   
...                                                  ...   
11796  A força normal é devida a forças repulsivas de...   
11821  onde é a área da seção transversal relevante p...   
11848  Para certos cenários físicos, é impossível mod...   
11849  Para certos cenários físicos, é impossível mod...   
11853  Para certos cenários físicos, é impossível mod...   

                                                   query  \
34     Com o que as tradições maternas dos nórdicos a...   
39     Quem é que a arristocracia acabou servindo com...   
55     Quem se juntou às forças normandas na destruiç...   
80                              Quem tr

In [1]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
from dbscan_methods import *

import warnings
# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

####faz sentido testar o hdbscan aqui?? muitos documentos variados que já estão separados pelo nome do documento


df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names_and_answers.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=12,embedding_column_name='embedding')
centroids = calculate_dbscan_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 4
query_text = "Como Aristóteles se referiu ao movimento forçado?"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)

best_chunks_info, best_similarities = find_chunks_in_clusters_dbscan(centroids, query_umap,  df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


Distance from query to centroid of cluster '127': 0.16482163965702057
Distance from query to centroid of cluster '133': 4.859380722045898
Distance from query to centroid of cluster '145': 5.43478536605835
Distance from query to centroid of cluster '152': 5.511082172393799
Distance from query to centroid of cluster '144': 6.149543762207031
Distance from query to centroid of cluster '150': 6.378132343292236
Distance from query to centroid of cluster '151': 6.43618631362915
Distance from query to centroid of cluster '154': 6.534212112426758
Distance from query to centroid of cluster '143': 6.979040622711182
Distance from query to centroid of cluster '159': 7.116640090942383
Distance from query to centroid of cluster '134': 7.148869514465332
Distance from query to centroid of cluster '136': 7.366012096405029
Distance from query to centroid of cluster '146': 7.378154754638672
Distance from query to centroid of cluster '129': 7.4630584716796875
Distance from query to centroid of cluster '126

KeyError: "None of [Index(['embedding 1', 'embedding 2', 'embedding 3', 'embedding 4',\n       'embedding 5', 'embedding 6', 'embedding 7', 'embedding 8',\n       'embedding 9', 'embedding 10', 'embedding 11', 'embedding 12'],\n      dtype='object')] are in the [index]"

In [1]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
from dbscan_methods import *
from meanshift_methods import *

import warnings
# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

####faz sentido testar o hdbscan aqui?? muitos documentos variados que já estão separados pelo nome do documento


df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)
umap, normalizer,n_components,df  = create_umap(df,n_components=512,embedding_column_name='embedding')
centroids = calculate_meanshift_centroids(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)
distance_threshold = 4
query_text = "Como Aristóteles se referiu ao movimento forçado?"
query_embedding = create_query_embedding(query_text)
query_umap = create_query_umap(query_embedding,umap,normalizer)

best_chunks_info, best_similarities = find_chunks_in_clusters_meanshift(centroids, query_umap,  df,distance_threshold=distance_threshold,embedding_column_name='embedding')
show_findings(best_chunks_info, best_similarities)

C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


Distance from query to centroid of cluster '0': 9.363401412963867
Searching for the best chunks in the following clusters: [0]


KeyError: "None of [Index(['embedding 1', 'embedding 2', 'embedding 3', 'embedding 4',\n       'embedding 5', 'embedding 6', 'embedding 7', 'embedding 8',\n       'embedding 9', 'embedding 10',\n       ...\n       'embedding 503', 'embedding 504', 'embedding 505', 'embedding 506',\n       'embedding 507', 'embedding 508', 'embedding 509', 'embedding 510',\n       'embedding 511', 'embedding 512'],\n      dtype='object', length=512)] are in the [index]"

In [4]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
from dbscan_methods import *
from meanshift_methods import *

import warnings
# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

####faz sentido testar o hdbscan aqui?? muitos documentos variados que já estão separados pelo nome do documento


df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)

print(df.head())

                                          chunk_text  \
0  Os normandos (normandos: nourmands; francês: n...   
1  Os normandos (normandos: nourmands; francês: n...   
2  Os normandos (normandos: nourmands; francês: n...   
3  Os normandos (normandos: nourmands; francês: n...   
4  Os normandos (normandos: nourmands; francês: n...   

                                               query  \
0           Em que país a Normandia está localizada?   
1          Quando os normandos estavam na Normandia?   
2         De quais países os nórdicos se originaram?   
3                          Quem era o líder nórdico?   
4  Em que século os normandos ganharam sua identi...   

                                           embedding document_name  
0  [0.02337769791483879, 0.06848494708538055, -0....       Normans  
1  [0.02337769791483879, 0.06848494708538055, -0....       Normans  
2  [0.02337769791483879, 0.06848494708538055, -0....       Normans  
3  [0.02337769791483879, 0.06848494708538055, -0..

In [3]:
import json
import pandas as pd

file_path = 'brquad-gte-dev-v2.0.json'

# Lendo os dados JSON do arquivo
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializa listas para armazenar os dados extraídos
document_names = []
context_texts = []
questions = []
answers = []

# Iterando sobre os dados para extrair as informações necessárias
for item in data['data']:
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            # Aqui assumimos que você está interessado apenas na primeira resposta encontrada
            answer = qa['answers'][0]['text'] if qa['answers'] else 'N/A'  # Lidar com questões sem resposta

            # Adiciona os dados extraídos às listas
            document_names.append(item['title'])
            context_texts.append(context)
            questions.append(question)
            answers.append(answer)

# Criando um DataFrame a partir das listas
df = pd.DataFrame({
    'document_name': document_names,
    'context': context_texts,
    'query': questions,
    'answer': answers,
    'embedding': embedding
})

print(df.head())

# Salva o DataFrame em um arquivo pickle para uso futuro
df.to_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names_and_answers.pkl')


  document_name                                            context  \
0       Normans  Os normandos (normandos: nourmands; francês: n...   
1       Normans  Os normandos (normandos: nourmands; francês: n...   
2       Normans  Os normandos (normandos: nourmands; francês: n...   
3       Normans  Os normandos (normandos: nourmands; francês: n...   
4       Normans  Os normandos (normandos: nourmands; francês: n...   

                                               query  \
0           Em que país a Normandia está localizada?   
1          Quando os normandos estavam na Normandia?   
2         De quais países os nórdicos se originaram?   
3                          Quem era o líder nórdico?   
4  Em que século os normandos ganharam sua identi...   

                          answer  
0                         França  
1                 Séculos X e XI  
2  Dinamarca, Islândia e Noruega  
3                          Rollo  
4                       Século X  


In [5]:
import json
import pandas as pd

# Assuming your existing DataFrame is named df and created from the earlier code
# Replace 'your_file_path.json' with the actual path to your JSON file
file_path = 'brquad-gte-dev-v2.0.json'

# Reading the JSON data from the file again to add the document names
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extracting document names (titles) for each context-question pair
answers = []

# Iterando sobre os dados para extrair as informações necessárias
for item in data['data']:
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            # Aqui assumimos que você está interessado apenas na primeira resposta encontrada
            answer = qa['answers'][0]['text'] if qa['answers'] else 'N/A'  # Lidar com questões sem resposta
            answers.append(answer)

# Assuming the lengths of df and document_names match
df['answer'] = answers

# Now df contains three columns: 'context', 'query', and 'document name'
print(df.head())

df.to_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names_and_answers.pkl')


                                          chunk_text  \
0  Os normandos (normandos: nourmands; francês: n...   
1  Os normandos (normandos: nourmands; francês: n...   
2  Os normandos (normandos: nourmands; francês: n...   
3  Os normandos (normandos: nourmands; francês: n...   
4  Os normandos (normandos: nourmands; francês: n...   

                                               query  \
0           Em que país a Normandia está localizada?   
1          Quando os normandos estavam na Normandia?   
2         De quais países os nórdicos se originaram?   
3                          Quem era o líder nórdico?   
4  Em que século os normandos ganharam sua identi...   

                                           embedding document_name  \
0  [0.02337769791483879, 0.06848494708538055, -0....       Normans   
1  [0.02337769791483879, 0.06848494708538055, -0....       Normans   
2  [0.02337769791483879, 0.06848494708538055, -0....       Normans   
3  [0.02337769791483879, 0.06848494708538055, 

In [1]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import PyPDF2  # Import PyPDF2
from search_functions import *
from plot_functions import *
from cluster_functions import *
from query_functions import *
from main import *
import pandas as pd
from umap_methods import *
from hdbscan_methods import *
from openai_utils import *
import warnings
# Suprimir apenas PerformanceWarning
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

####faz sentido testar o hdbscan aqui?? muitos documentos variados que já estão separados pelo nome do documento


df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
#df['embeddings_summary'] = df['embeddings_summary'].apply(lambda x: x.data[0].embedding)



pca, normalizer,n_components,df  = create_pca(df,n_components=32,embedding_column_name='embedding')
centroids = calculate_hdbscan_centroids_with_pca(df,n_components=n_components)
#df,fig,centroids = plot_clusters_2_componentsn(df,plot=False)


distance_threshold = 1
query_text = "Como Aristóteles se referiu ao movimento forçado?"
query_embedding = create_query_embedding(query_text)
query_pca = create_query_pca(pca,normalizer,query_embedding)

#ok até aqui

best_chunks_info, best_similarities = find_chunks_in_multiple_documents_euclidean(centroids, query_pca,query_embedding,  df,
                                                                                  distance_threshold=distance_threshold,
                                                                                  embedding_column_name='embedding',print_distances=False)
show_findings(best_chunks_info, best_similarities)
contexts = [chunk['chunk_text'] for chunk in best_chunks_info]
response = ask_with_openai(contexts[0],query_text)
print("Pergunta: ",query_text)
print("Resposta: ",response)   


C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [6]:
from test_methods import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
query_text = "Como Aristóteles se referiu ao movimento forçado?"

df,centroids_parameters = return_centroids_parameters_hdbscan(df,n_components=128,embedding_column_name='embedding')
best_context = return_best_context_centroids(df, query_text, centroids_parameters,distance_threshold = 5,show_results = True)
response = ask_with_openai(best_context,query_text)
print("Pergunta: ",query_text)
print("Resposta: ",response) 


                      Best Chunk Infos                      
Chunk #1:
Best Similarity:    0.6515924911990193
Document Name:      Força
Cluster:            434
Cluster:            Força
Chunk Text:         Aristóteles forneceu uma discussão filosófica do conceito de força como parte integrante da cosmologia aristotélica. Na visão de Aristóteles, a esfera terrestre continha quatro elementos que repousam em diferentes "lugares naturais" nela. Aristóteles acreditava que objetos imóveis na Terra, compostos principalmente dos elementos terra e água, estavam em seu lugar natural no chão e que permaneceriam assim se deixados em paz. Ele distinguiu entre a tendência inata dos objetos para encontrar seu "lugar natural" (por exemplo, a queda de corpos pesados), o que levou ao "movimento natural" e o movimento não natural ou forçado, que exigia a aplicação contínua de uma força. Essa teoria, baseada na experiência cotidiana de como os objetos se movem, como a aplicação constante de uma força nece

In [1]:
from test_methods import *
df = pd.read_pickle('datasets/brquad-gte-dev-v2.0_embedded_with_document_names.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
query_text = "Como Aristóteles se referiu ao movimento forçado?"
df,centroids_parameters = return_centroids_parameters(df,n_components=128,embedding_column_name='embedding')
best_context = return_best_context_centroids(df, query_text, centroids_parameters,distance_threshold = 5,show_results = True)
response = ask_with_openai(best_context,query_text)
print("Pergunta: ",query_text)
print("Resposta: ",response) 


C:\Users\Cliente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                              chunk_text  \
11496  Filósofos da antiguidade usavam o conceito de ...   
11497  Filósofos da antiguidade usavam o conceito de ...   
11498  Filósofos da antiguidade usavam o conceito de ...   
11499  Filósofos da antiguidade usavam o conceito de ...   
11500  Filósofos da antiguidade usavam o conceito de ...   

                                                   query  \
11496  Que conceito os filósofos da antiguidade usava...   
11497  Qual era a crença de que manter o movimento ex...   
11498                Quem possuía matemática matemática?   
11499  Quanto tempo levou para melhorar as leis de mo...   
11500         Quem desenvolveu a teoria da relatividade?   

                                               embedding document_name  \
11496  [-0.0065753585658967495, 0.052408043295145035,...         Força   
11497  [-0.006602620240300894, 0.05243142694234848, -...         Força   
11498  [-0.0065753585658967495, 0.052408043295145035,...

In [5]:
import pandas as pd

df = pd.read_pickle('results_df_precision_1.00_datetime_20240405_155033.pkl')
df.to_csv('logs/results_df_precision_1.00_datetime_20240405_155033.csv',index=False)


In [7]:
import pandas as pd

df = pd.read_pickle('results_df_precision_1.00_datetime_20240405_195206.pkl')
df.to_csv('logs/results_df_precision_1.00_datetime_20240405_195206.csv',index=False)
